# LSTM Stock Predictor Using Fear and Greed Index

In this notebook, you will build and train a custom LSTM RNN that uses a 10 day window of Bitcoin fear and greed index values to predict the 11th day closing price. 

You will need to:

1. Prepare the data for training and testing
2. Build and train a custom LSTM RNN
3. Evaluate the performance of the model

## Data Preparation

In this section, you will need to prepare the training and testing data for the model. The model will use a rolling 10 day window to predict the 11th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [31]:
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

In [32]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [33]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42


In [34]:
# Load the historical closing prices for Bitcoin
df2 = pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

Date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [35]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [36]:
df.head()

,fng_value,Close
2018-02-01,30,9114.719727
2018-02-02,15,8870.820313
2018-02-03,40,9251.269531
2018-02-04,24,8218.049805
2018-02-05,11,6937.080078


In [37]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [38]:
# Predict Closing Prices using a 10 day window of fear and greed index values and a target of the 11th day closing price
# Try a window size anywhere from 1 to 10 and see how the model performance changes
window_size = 1

# Column index 1 is the `Close` column
feature_column = 0
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

In [39]:
# Use 70% of the data for training and the remainder for testing
split_length = int(len(X) * 0.7)
X_train = X[:split_length]
X_test = X[split_length:]
y_train = y[:split_length]
y_test = y[split_length:]
X_train[:5]

array([[30, 15, 40, 24, 11,  8, 36, 30, 44, 54],
       [15, 40, 24, 11,  8, 36, 30, 44, 54, 31],
       [40, 24, 11,  8, 36, 30, 44, 54, 31, 42],
       [24, 11,  8, 36, 30, 44, 54, 31, 42, 35],
       [11,  8, 36, 30, 44, 54, 31, 42, 35, 55]])

In [40]:
# Use MinMaxScaler to scale the data between 0 and 1. 
train_and_test_arrays = [X_train, X_test, y_train, y_test]
new_arrays = []
for array in train_and_test_arrays:
    scaler = MinMaxScaler()
    array = scaler.fit_transform(array)
    new_arrays.append(array)
X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = new_arrays   
X_train_scaled[:5]

array([[0.33333333, 0.10606061, 0.48484848, 0.24242424, 0.04545455,
        0.        , 0.41538462, 0.32307692, 0.53846154, 0.69230769],
       [0.10606061, 0.48484848, 0.24242424, 0.04545455, 0.        ,
        0.42424242, 0.32307692, 0.53846154, 0.69230769, 0.33846154],
       [0.48484848, 0.24242424, 0.04545455, 0.        , 0.42424242,
        0.33333333, 0.53846154, 0.69230769, 0.33846154, 0.50769231],
       [0.24242424, 0.04545455, 0.        , 0.42424242, 0.33333333,
        0.54545455, 0.69230769, 0.33846154, 0.50769231, 0.4       ],
       [0.04545455, 0.        , 0.42424242, 0.33333333, 0.54545455,
        0.6969697 , 0.33846154, 0.50769231, 0.4       , 0.70769231]])

In [41]:
# Reshape the features for the model
X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))
X_train_scaled[:5]

array([[[0.33333333],
        [0.10606061],
        [0.48484848],
        [0.24242424],
        [0.04545455],
        [0.        ],
        [0.41538462],
        [0.32307692],
        [0.53846154],
        [0.69230769]],

       [[0.10606061],
        [0.48484848],
        [0.24242424],
        [0.04545455],
        [0.        ],
        [0.42424242],
        [0.32307692],
        [0.53846154],
        [0.69230769],
        [0.33846154]],

       [[0.48484848],
        [0.24242424],
        [0.04545455],
        [0.        ],
        [0.42424242],
        [0.33333333],
        [0.53846154],
        [0.69230769],
        [0.33846154],
        [0.50769231]],

       [[0.24242424],
        [0.04545455],
        [0.        ],
        [0.42424242],
        [0.33333333],
        [0.54545455],
        [0.69230769],
        [0.33846154],
        [0.50769231],
        [0.4       ]],

       [[0.04545455],
        [0.        ],
        [0.42424242],
        [0.33333333],
        [0.54545455],
  

---

## Build and Train the LSTM RNN

In this section, you will design a custom LSTM RNN and fit (train) it using the training data.

You will need to:
1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

### Hints:
You will want to use the same model architecture and random seed for both notebooks. This is necessary to accurately compare the performance of the FNG model vs the closing price model. 

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [43]:
# Build the LSTM model. 
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
model = Sequential()
number_units = 80
dropout_fraction = 0.30
# Layer 1
model.add(LSTM(
    units = number_units,
    return_sequences = True,
    input_shape = (X_train_scaled.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))
# Layer 2
model.add(LSTM(units = number_units, return_sequences = True))
model.add(Dropout(dropout_fraction))
# Layer 3
model.add(LSTM(units = number_units))
model.add(Dropout(dropout_fraction))
# Output Layer
model.add(Dense(1))

In [44]:
# Compile the model
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [45]:
# Summarize the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10, 80)            26240     
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 80)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 10, 80)            51520     
_________________________________________________________________
dropout_4 (Dropout)          (None, 10, 80)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 80)                51520     
_________________________________________________________________
dropout_5 (Dropout)          (None, 80)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

In [46]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended
model.fit(X_train_scaled,y_train_scaled, epochs = 20, shuffle = False, batch_size = 3, verbose = 1)

Epoch 1/20
124/124 [==============================] - 1s 7ms/step - loss: 0.0422
Epoch 2/20
124/124 [==============================] - 1s 8ms/step - loss: 0.0544
Epoch 3/20
124/124 [==============================] - 1s 8ms/step - loss: 0.0574
Epoch 4/20
124/124 [==============================] - 1s 8ms/step - loss: 0.0606
Epoch 5/20
124/124 [==============================] - 1s 8ms/step - loss: 0.0636
Epoch 6/20
124/124 [==============================] - 1s 7ms/step - loss: 0.0640
Epoch 7/20
124/124 [==============================] - 1s 7ms/step - loss: 0.0637
Epoch 8/20
124/124 [==============================] - 1s 7ms/step - loss: 0.0626
Epoch 9/20
124/124 [==============================] - 1s 7ms/step - loss: 0.0601
Epoch 10/20
124/124 [==============================] - 1s 8ms/step - loss: 0.0619
Epoch 11/20
124/124 [==============================] - 1s 8ms/step - loss: 0.0591
Epoch 12/20
124/124 [==============================] - 1s 8ms/step - loss: 0.0585
Epoch 13/20
124/124 [====

---

## Model Performance

In this section, you will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [47]:
# Evaluate the model
model.evaluate(X_test_scaled, y_test_scaled)

5/5 [==============================] - 0s 4ms/step - loss: 0.0887


0.08873828500509262

In [48]:
# Make some predictions
predicted = model.predict(X_test_scaled)

In [49]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [50]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
})
stocks.head()

,Real,Predicted
0,3.603610e+07,6057.014160
1,3.649346e+07,6200.673828
2,3.615363e+07,6283.832031
3,3.658050e+07,6332.154785
4,3.810068e+07,6368.447266


In [51]:
# Plot the real vs predicted values as a line chart
stocks.hvplot(title = 'Real vs Predicted', xlabel = 'Days', ylabel = 'Price')

:NdOverlay   [Variable]
   :Curve   [index]   (value)